# Trabalho 2 de IA

* Bruno Alvarenga Colturato,   11200251
* Che Fan Pan,                 11200421
* Eduardo Cavalari Valença,    11234381

In [ ]:
import pandas
import seaborn
import numpy as np

In [ ]:
datasets_names = ['cs', 
            'breast_cancer', 
            'diabetes', 
            'driving_behavior',
            'employee',
            'go_to_college',
            'phishing',
            'smoking',
            'social_network_ads',
            'water_quality']

datasets_files = list(map(lambda x : 'datasets/' + x + '.csv', datasets_names))
datasets = [None] * 10

# Pré-processamento

### CS:GO Round Winner Classification - dataset[0]

In [ ]:
# Load data and drop empty rows
datasets[0] = pandas.read_csv(datasets_files[0])
datasets[0] = datasets[0].dropna(axis=0)

# Transform categorical data into many binary data 
datasets[0] = pandas.get_dummies(datasets[0], columns=["map"])

# Define target column
datasets[0] = datasets[0].rename(columns={'round_winner' : 'target'})

datasets[0]

### Breast Cancer - dataset[1]

In [ ]:
# Load data and drop empty rows
datasets[1] = pandas.read_csv(datasets_files[1])
datasets[1] = datasets[1].dropna(axis=0)

# Drop useless column
datasets[1] = datasets[1].drop(columns=['id'])

# Define target column
datasets[1] = datasets[1].rename(columns={'diagnosis' : 'target'})

datasets[1]

### Diabetes - dataset[2]

In [ ]:
# Load data and drop empty rows
datasets[2] = pandas.read_csv(datasets_files[2])
datasets[2] = datasets[2].dropna(axis=0)

# Transform categorical data into many binary data 
columns2 = datasets[2].columns.values.tolist()
columns2.remove('Diabetic')
datasets[2] = pandas.get_dummies(datasets[2], columns=columns2)

# Define target column
datasets[2] = datasets[2].rename(columns={'Diabetic' : 'target'})

datasets[2]

### Driving behavior - dataset[3]

In [ ]:
# Load data and drop empty rows
datasets[3] = pandas.read_csv(datasets_files[3])
datasets[3] = datasets[3].dropna(axis=0)

# Drop useless column
datasets[3] = datasets[3].drop(columns=['Timestamp'])

# Define target column
datasets[3] = datasets[3].rename(columns={'Class' : 'target'})

datasets[3]

### Employee - dataset[4]

In [ ]:
# Load data and drop empty rows
datasets[4] = pandas.read_csv(datasets_files[4])
datasets[4] = datasets[4].dropna(axis=0)

# Transform categorical data into many binary data 
datasets[4] = pandas.get_dummies(datasets[4])

# Define target column
datasets[4] = datasets[4].rename(columns={'LeaveOrNot' : 'target'})

datasets[4]

### Go to college - dataset[5]

In [ ]:
# Load data and drop empty rows
datasets[5] = pandas.read_csv(datasets_files[5])
datasets[5] = datasets[5].dropna(axis=0)

# Transform categorical data into many binary data 
datasets[5] = pandas.get_dummies(datasets[5])

# Define target column
datasets[5] = datasets[5].rename(columns={'in_college' : 'target'})

datasets[5]["target"] = datasets[5]["target"].astype('uint8')

datasets[5]

### Phishing - dataset[6]

In [ ]:
# Load data and drop empty rows
datasets[6] = pandas.read_csv(datasets_files[6])
datasets[6] = datasets[6].dropna(axis=0)

# Transform categorical data into many binary data 
datasets[6] = pandas.get_dummies(datasets[6])

# Drop useless column
datasets[6] = datasets[6].drop(columns=['Index'])

# Define target column
datasets[6] = datasets[6].rename(columns={'class' : 'target'})
datasets[6]["target"].replace(to_replace = -1, value = 0, inplace=True)

datasets[6]

### Smoking - dataset[7]

In [ ]:
# Load data and drop empty rows
datasets[7] = pandas.read_csv(datasets_files[7])
datasets[7] = datasets[7].dropna(axis=0)

# Transform categorical data into many binary data 
datasets[7] = pandas.get_dummies(datasets[7])

# Drop useless column
datasets[7] = datasets[7].drop(columns=['ID'])

# Define target column
datasets[7] = datasets[7].rename(columns={'smoking' : 'target'})

datasets[7]

### Social network ads - dataset[8]

In [ ]:

# Load data and drop empty rows
datasets[8] = pandas.read_csv(datasets_files[8])
datasets[8] = datasets[8].dropna(axis=0)

# Transform categorical data into many binary data 
datasets[8] = pandas.get_dummies(datasets[8])

# Drop useless column
datasets[8] = datasets[8].drop(columns=['User ID'])

# Define target column
datasets[8] = datasets[8].rename(columns={'Purchased' : 'target'})

datasets[8]

### Water quality - dataset[9]

In [ ]:
# Load data and drop empty rows
datasets[9] = pandas.read_csv(datasets_files[9])
datasets[9] = datasets[9].dropna(axis=0)

# Define target column
datasets[9] = datasets[9].rename(columns={'is_safe' : 'target'})

datasets[9]

# Análise de dados

In [ ]:
# All columns except target
X = [None] * 10
# Target column
Y = [None] * 10

for i, dataset in enumerate(datasets):
    X[i] = dataset.loc[:, dataset.columns != 'target'] 
    Y[i] = np.array(dataset.loc[:, dataset.columns == 'target']).ravel()

In [ ]:
import matplotlib.pyplot as plt

# Type of evaluation
scoring_choosed = [None] * 10

for i, dataset in enumerate(datasets):
    # Para plotar a proporção das classes, primeiro contamos os valores da coluna 'target' e transformamos em um frame
    class_distribuition = pandas.DataFrame(dataset['target'].value_counts())
    
    plt.figure()

    # Usamos o frame criado acima para plot a frequencia de cada classe no dataset
    plot = seaborn.barplot(x=class_distribuition.index, y='target', data = class_distribuition)
    plot.set_title(datasets_names[i])

    total_sum = sum(class_distribuition['target'])
    partition = total_sum/len(class_distribuition['target'])
    for value in class_distribuition['target']:
        if(0.9*partition <= value <= 1.1*partition):
            scoring_choosed[i] = "accuracy" 
        else:
            scoring_choosed[i] = "balanced_accuracy"
            break

In [ ]:
# importa metodos de classicacao
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# instancia os classificador 
DTC = DecisionTreeClassifier()
KNN = KNeighborsClassifier()
GNB = GaussianNB()
MLP = MLPClassifier()

In [ ]:
DTC_parameters = {'criterion' : ['gini','entropy','log_loss'], 'random_state' : [42]}
KNN_parameters = {'n_neighbors' : [1, 3, 5, 7, 9, 11, 13], 'weights' : ['uniform', 'distance']}
GNB_parameters = {}
MLP_parameters = {'hidden_layer_sizes' : [(5), (8), (15), (5, 3), (8, 5), (10, 5)], 'max_iter' : [3000], 'random_state' : [42]}

In [ ]:
GS_DTC = [None] * 10
GS_KNN = [None] * 10
GS_GNB = [None] * 10
GS_MLP = [None] * 10

for i, choosed in enumerate(scoring_choosed):
    GS_DTC[i] = GridSearchCV(DTC, DTC_parameters, cv = 10, scoring = choosed)
    GS_KNN[i] = GridSearchCV(KNN, KNN_parameters, cv = 10, scoring = choosed)
    GS_GNB[i] = GridSearchCV(GNB, GNB_parameters, cv = 10, scoring = choosed)
    GS_MLP[i] = GridSearchCV(MLP, MLP_parameters, cv = 10, scoring = choosed)

In [ ]:
# treina o classificador com o conjunto de treino
for i in range(len(datasets)):
    GS_DTC[i].fit(X[i], Y[i])
    GS_KNN[i].fit(X[i], Y[i])
    GS_GNB[i].fit(X[i], Y[i])
    GS_MLP[i].fit(X[i], Y[i])


In [ ]:
results_DTC = [None] * 10
results_KNN = [None] * 10
results_GNB = [None] * 10
results_MLP = [None] * 10

for i in range(len(datasets)):
    results_DTC[i] = pandas.DataFrame(GS_DTC[i].cv_results_)
    results_KNN[i] = pandas.DataFrame(GS_KNN[i].cv_results_)
    results_GNB[i] = pandas.DataFrame(GS_GNB[i].cv_results_)
    results_MLP[i] = pandas.DataFrame(GS_MLP[i].cv_results_)

In [ ]:
view = ['params', 'mean_test_score','std_test_score', 'rank_test_score']

for i in range(len(datasets)):
    results_DTC[i] = results_DTC[i][view].sort_values(by='rank_test_score')
    results_KNN[i] = results_KNN[i][view].sort_values(by='rank_test_score')
    results_GNB[i] = results_GNB[i][view].sort_values(by='rank_test_score')
    results_MLP[i] = results_MLP[i][view].sort_values(by='rank_test_score')

In [ ]:
for i, dataset in enumerate(datasets_names):
    print(f'Para o dataset {dataset}:')
    print(f'\tA acurácia do Classificador DTC é: {results_DTC[i]["mean_test_score"][0] :.2%}')
    print(f'\tA acurácia do Classificador KNN é: {results_KNN[i]["mean_test_score"][0] :.2%}')
    print(f'\tA acurácia do Classificador GNB é: {results_GNB[i]["mean_test_score"][0] :.2%}')
    print(f'\tA acurácia do Classificador MLP é: {results_MLP[i]["mean_test_score"][0] :.2%}')
    print('\n')

In [ ]:

datasets_algs = [[] for _ in range(len(datasets))]

for i, algs in enumerate(datasets_algs):
    algs.append({"name": "DTC", "results": results_DTC[i]})
    algs.append({"name": "KNN", "results": results_KNN[i]})
    algs.append({"name": "GNB", "results": results_GNB[i]})
    algs.append({"name": "MLP", "results": results_MLP[i]})
    algs.sort(key = lambda acc : acc["results"]["mean_test_score"][0], reverse = True)

    print(f'Dataset: {datasets_names[i]}')
    for alg in algs:
        print(f'\t{alg["name"]}, media: {alg["results"]["mean_test_score"][0]}, dp: {alg["results"]["std_test_score"][0]}')


In [ ]:
from math import sqrt

diferenca_absoluta = [None] * 10

for i, algs in enumerate(datasets_algs):
    media_err_A0 = 1 - algs[0]["results"]["mean_test_score"][0]
    dp_A0 = algs[0]["results"]["std_test_score"][0]

    media_err_A1 = 1 - algs[1]["results"]["mean_test_score"][0]
    dp_A1 = algs[1]["results"]["std_test_score"][0]

    dif_media_err = media_err_A0 - media_err_A1
    desvio_padrao = sqrt( (dp_A0**2 + dp_A1**2) / 2.0 )
    diferenca_absoluta[i] = dif_media_err / desvio_padrao

In [ ]:
for i, algs in enumerate(datasets_algs):
    print(f'Dataset: {datasets_names[i]}, diferenca_absoluta: {diferenca_absoluta[i]}')
    if diferenca_absoluta[i] > 0:
        if diferenca_absoluta[i] >= 0.02:
            print(f'{algs[1]["name"]} tem melhor performance que {algs[0]["name"]} com nível de confiança de 95%')
        else:
            print(f'{algs[1]["name"]} tem melhor performance que {algs[0]["name"]}')
    else:
        if diferenca_absoluta[i] <= 0.02:
            print(f'{algs[0]["name"]} tem melhor performance que {algs[1]["name"]} com nível de confiança de 95%')
        else:
            print(f'{algs[0]["name"]} tem melhor performance que {algs[1]["name"]}')